In [2]:
import cv2

# Load the cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Read the input image
img = cv2.imread('OIP.jpeg')

# Convert into grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Detect faces
faces = face_cascade.detectMultiScale(gray, 1.1, 4)

# Draw rectangle around the faces
for (x, y, w, h) in faces:
    cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)

# Display the output
cv2.imshow('img', img)
cv2.waitKey()

-1

In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc


# Step 1: Reshape images to 1D vectors
def reshape_images(images):
    return images.reshape(images.shape[0], -1).T

# Step 2: Construct data matrix
def construct_data_matrix(images):
    return reshape_images(images)

# Step 3: Get the Mean Image
def get_mean_image(data_matrix):
    num_images = data_matrix.shape[1]
    mean_image = np.sum(data_matrix, axis=1, keepdims=True) / num_images
    return mean_image

# Step 4: Subtract the mean image from all images
def subtract_mean_image(data_matrix, mean_image):
    return data_matrix - mean_image

# Step 5: Get the Covariance matrix
def get_covariance_matrix(subtracted_data):
    num_images = subtracted_data.shape[1]
    covariance_matrix = np.dot(subtracted_data, subtracted_data.T) / num_images
    return covariance_matrix

# Step 6: Normalize the eigen vectors
def normalize_eigen_vectors(eigen_values, eigen_vectors, threshold=0.9):
    total_var = np.sum(eigen_values)
    var_sum = 0
    sorted_indices = np.argsort(eigen_values)[::-1]
    for i, idx in enumerate(sorted_indices):
        var_sum += eigen_values[idx]
        if var_sum / total_var >= threshold:
            return eigen_vectors[:, sorted_indices[:i + 1]]

# Step 7: Keep only the significant eigen vectors
def keep_significant_eigen_vectors(eigen_values, eigen_vectors, threshold=0.9):
    normalized_eigen_vectors = normalize_eigen_vectors(eigen_values, eigen_vectors, threshold)
    return normalized_eigen_vectors

# Step 8: Map all images to new components
def map_to_new_components(data_matrix, eigen_vectors):
    return np.dot(eigen_vectors.T, data_matrix)

# Step 9: Face Recognition
def recognize_face(image, mean_image, eigen_vectors, projected_data, threshold):
    image = image.reshape(-1, 1)
    mean_image = mean_image.reshape(-1, 1)  # Ensure mean image has the same shape as the test image

    subtracted_image = image - mean_image
    projected_image = np.dot(eigen_vectors.T, subtracted_image)
    distances = np.linalg.norm(projected_data - projected_image, axis=0)
    recognized_index = np.argmin(distances)
    min_distance = distances[recognized_index]
    if min_distance > threshold:
        return None, min_distance
    return recognized_index, min_distance

# Function to load images from a directory
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename), cv2.IMREAD_GRAYSCALE)
        if img is not None:
            images.append(img)
    return np.array(images)

# Function to perform face recognition
def perform_face_recognition(images_folder, test_image, threshold=3500):
    # Load images
    images = load_images_from_folder(images_folder)
    
    if len(images) == 0:
        return None, None, None, None, "No images in the dataset"
    
    # Construct data matrix
    data_matrix = construct_data_matrix(images)
    
    # Get mean image
    mean_image = get_mean_image(data_matrix)
    
    # Subtract the mean image from all images
    subtracted_data = subtract_mean_image(data_matrix, mean_image)
    
    # Get the covariance matrix
    covariance_matrix = get_covariance_matrix(subtracted_data)
    
    # Get eigen values and eigen vectors
    eigen_values, eigen_vectors = np.linalg.eig(covariance_matrix)
    
    # Keep only the significant eigen vectors
    eigen_vectors = keep_significant_eigen_vectors(eigen_values, eigen_vectors)
    
    # Map all images to new components
    projected_data = map_to_new_components(subtracted_data, eigen_vectors)
    
    # Load test image
    # test_image = cv2.imread(test_image_path, cv2.IMREAD_GRAYSCALE)
    
    if test_image is None:
        return None, None, None, None, "Test image not found"
    
    # Recognize face
    recognized_index, distance = recognize_face(test_image, mean_image, eigen_vectors, projected_data, threshold)
    
    if recognized_index is None:
        return None, None, None, None, "Test image not similar to the dataset"
    
    # Get the name of the recognized image
    recognized_image_name = os.listdir(images_folder)[recognized_index]
    
    # Load recognized image
    recognized_image = images[recognized_index]
    
    # Return recognized index, distance, test image, and recognized image name
    return recognized_index, distance, test_image, recognized_image, recognized_image_name

# Test the face recognition system
recognized_index, distance, test_image, recognized_image, recognized_image_name = perform_face_recognition("MyProjectsProgress_Face_Detection_and_Recognition_main_faces (1)", "5-9.bmp")

if recognized_index is not None:
    print("Recognized Image Name:", recognized_image_name)
    print("Distance:", distance)
    # Display test image, recognized image, and recognition result
    plt.subplot(1, 2, 1)
    plt.imshow(test_image, cmap='gray')
    plt.title('Test Image')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(recognized_image, cmap='gray')
    plt.title('Recognized Image')
    plt.axis('off')

    plt.suptitle(f'Recognized: {recognized_index} - Distance: {distance:.2f}')
    plt.show()
else:
    print("Test image not similar to the dataset")
# cal the roc curve for more than one images 


def ROC_curve(recognized_index, distance, test_image, recognized_image, recognized_image_name):
    # Calculate the threshold
    threshold = 0.01 * distance

    # Calculate the true positive rate (TPR) and false positive rate (FPR)
    tpr = np.sum(recognized_index == recognized_index) / len(recognized_index)
    fpr = np.sum(distance < threshold) / len(distance)

    # Return the TPR and FPR
    return tpr, fpr
#draw roc curve
def draw_roc_curve(tpr, fpr):
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.show()




In [ ]:
def calculate_roc_curve(images_folder, threshold=3500):
    # Load images
    images = load_images_from_folder(images_folder)
    
    if len(images) == 0:
        return None, None, None, "No images in the dataset"
    
    # Construct data matrix
    data_matrix = construct_data_matrix(images)
    
    # Get mean image
    mean_image = get_mean_image(data_matrix)
    
    # Subtract the mean image from all images
    subtracted_data = subtract_mean_image(data_matrix, mean_image)
    
    # Get the covariance matrix
    covariance_matrix = get_covariance_matrix(subtracted_data)
    
    # Get eigen values and eigen vectors
    eigen_values, eigen_vectors = np.linalg.eig(covariance_matrix)
    
    # Keep only the significant eigen vectors
    eigen_vectors = keep_significant_eigen_vectors(eigen_values, eigen_vectors)
    
    # Map all images to new components
    projected_data = map_to_new_components(subtracted_data, eigen_vectors)
    
    # Initialize true and false positive rates
    true_positive_rates = []
    false_positive_rates = []
    
    # Loop through each image in the dataset
    for i, image in enumerate(images):
        # Recognize face
        recognized_index, distance = recognize_face(image, mean_image, eigen_vectors, projected_data, threshold)
        
        if recognized_index is not None:
            true_positive_rates.append(1)
        else:
            true_positive_rates.append(0)
        
        # Loop through each image in the dataset again
        for j, other_image in enumerate(images):
            if i == j:
                continue
            # Recognize face
            recognized_index, distance = recognize_face(other_image, mean_image, eigen_vectors, projected_data, threshold)
            
            if recognized_index is not None:
                false_positive_rates.append(1)
            else:
                false_positive_rates.append(0)
    
    # Calculate ROC curve
    fpr, tpr, _ = roc_curve(true_positive_rates, false_positive_rates)
    roc_auc = auc(fpr, tpr)
    
    return fpr, tpr, roc_auc
#draw roc curve
fpr, tpr, roc_auc = calculate_roc_curve("MyProjectsProgress_Face_Detection_and_Recognition_main_faces (1)")
#draw roc curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()

In [10]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename), cv2.IMREAD_GRAYSCALE)
        if img is not None:
            images.append(img)
    return np.array(images)
# Calculate distances between the test image and other images
def calculate_distances(test_image, other_images):
    distances = []
    for image in other_images:
        distance = np.linalg.norm(test_image - image)
        distances.append(distance)
    return distances

# Generate ROC curve
def generate_roc_curve(distances, true_labels):
    fpr, tpr, thresholds = roc_curve(true_labels, distances)
    roc_auc = auc(fpr, tpr)
    return fpr, tpr, roc_auc

# Example data
test_image = cv2.imread('5-9.bmp', 0)  # Example test image
other_images = load_images_from_folder("MyProjectsProgress_Face_Detection_and_Recognition_main_faces (1)")  # Example other images
true_labels = np.array([1, 0, 0])  # Example true labels (1 for positive, 0 for negative)

# Calculate distances
distances = calculate_distances(test_image, other_images)

# Generate ROC curve
fpr, tpr, roc_auc = generate_roc_curve(distances, true_labels)

# Plot ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()


ValueError: Found input variables with inconsistent numbers of samples: [3, 45]